# Models Testing


In [1]:
import time

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.preprocessing import StandardScaler
from sklearn.utils import shuffle

import torch
import torch.nn as nn
from torch.autograd import Variable
from torch.utils.data import Dataset, DataLoader

In [2]:
# Set seeds
torch.manual_seed(0)
np.random.seed(0)

In [3]:
synthetic_calls_path = '../data/trinomial_synthetic_calls.csv'
synthetic_puts_path = '../data/trinomial_synthetic_puts.csv'

In [4]:
def reduce_mem_usage(df):
    """ iterate through all the columns of a dataframe and modify the data type
        to reduce memory usage.        
    """    
    for col in df.columns:
        col_type = df[col].dtype
        
        if col_type != object:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
        else:
            df[col] = df[col].astype('category')
    
    return df

In [5]:
synthetic_calls = pd.read_csv(synthetic_calls_path, index_col=0)
synthetic_puts = pd.read_csv(synthetic_puts_path, index_col=0)

synthetic_calls = reduce_mem_usage(synthetic_calls)
synthetic_puts = reduce_mem_usage(synthetic_puts)

In [6]:
synthetic_options = pd.concat([synthetic_calls, synthetic_puts], axis=0)
synthetic_options = shuffle(synthetic_options, random_state=0)
synthetic_options = synthetic_options.reset_index()
synthetic_options = synthetic_options.drop('index', axis=1)

# Preprocessing

In [7]:
synthetic_options = pd.get_dummies(synthetic_options, prefix='', prefix_sep='')

In [8]:
input_sc = StandardScaler()
output_sc = StandardScaler()
input_data = input_sc.fit_transform(synthetic_options.drop('Option Price', axis=1))
output_data = output_sc.fit_transform(synthetic_options['Option Price'].values.reshape(-1, 1))

train_size = 0.8
val_size = 0.1

last_train_idx = int(np.round(len(input_data) * train_size))
last_val_idx = last_train_idx + int(np.round(len(input_data) * val_size))

X_train = input_data[0:last_train_idx]
X_val = input_data[last_train_idx:last_val_idx]
X_test = input_data[last_val_idx:]

y_train = output_data[0:last_train_idx]
y_val = output_data[last_train_idx:last_val_idx]
y_test = output_data[last_val_idx:]

In [9]:
X_train = Variable(torch.Tensor(X_train))
X_val = Variable(torch.Tensor(X_val))
X_test = Variable(torch.Tensor(X_test))

y_train = Variable(torch.Tensor(y_train))
y_val = Variable(torch.Tensor(y_val))
y_test = Variable(torch.Tensor(y_test))

# Model

In [10]:
CUDA = torch.cuda.is_available()
device = 'cuda:0' if CUDA else 'cpu'

In [11]:
class ResBlock(nn.Module):

  def __init__(self, module):
    super(ResBlock, self).__init__()
    self.module = module

  def forward(self, x):
    return self.module(x) + x

In [12]:
class HiddenLayer(nn.Module):

  def __init__(self, layer_size, act_fn):
      super(HiddenLayer, self).__init__()
      
      if act_fn == 'ReLU':
        self.layer = nn.Sequential(
          nn.Linear(layer_size, layer_size),
          nn.ReLU())
      elif act_fn == 'LeakyReLU':
        self.layer = nn.Sequential(
          nn.Linear(layer_size, layer_size),
          nn.LeakyReLU())
      elif act_fn == 'ELU':
        self.layer = nn.Sequential(
          nn.Linear(layer_size, layer_size),
          nn.ELU())
    
  def forward(self, x):
    return self.layer(x)

In [13]:
class Net(nn.Module):

  def __init__(self, input_size, output_size, hidden_size, num_layers, act_fn):
    super(Net, self).__init__()
    self.input_size = input_size
    self.output_size = output_size
    self.hidden_size = hidden_size

    if act_fn == 'ReLU':
      self.initial_layer = nn.Sequential(
          nn.Linear(self.input_size, self.hidden_size),
          nn.ReLU())
    elif act_fn == 'LeakyReLU':
      self.initial_layer = nn.Sequential(
          nn.Linear(self.input_size, self.hidden_size),
          nn.LeakyReLU())
    elif act_fn == 'ELU':
      self.initial_layer = nn.Sequential(
          nn.Linear(self.input_size, self.hidden_size),
          nn.ELU())

    self.hidden_layers_list = []

    for i in range(num_layers // 2):
      self.hidden_layers_list.append(
          ResBlock(
            nn.Sequential(
                HiddenLayer(self.hidden_size, act_fn),
                HiddenLayer(self.hidden_size, act_fn)
            )
        )
      )

    self.hidden_layers = nn.Sequential(*self.hidden_layers_list)

    self.net = nn.Sequential(
        self.initial_layer,
        self.hidden_layers,
        nn.Linear(self.hidden_size, self.output_size)
    )
  
  def forward(self, x):
    return self.net(x)

In [14]:
def init_weights(m, init_m: str):

  @torch.no_grad()
  def init_uniform(m):
    if isinstance(m, nn.Linear):
      torch.nn.init.uniform_(m.weight)
      m.bias.data.fill_(0.01)

  @torch.no_grad()
  def init_normal(m):
    if isinstance(m, nn.Linear):
      torch.nn.init.normal_(m.weight)
      m.bias.data.fill_(0.01)

  @torch.no_grad()
  def init_xuniform(m):
    if isinstance(m, nn.Linear):
      torch.nn.init.xavier_uniform_(m.weight)
      m.bias.data.fill_(0.01)

  @torch.no_grad()
  def init_xnormal(m):
    if isinstance(m, nn.Linear):
      torch.nn.init.xavier_normal_(m.weight)
      m.bias.data.fill_(0.01)

  if init_m == 'uniform':
    m.apply(init_uniform)
  elif init_m == 'normal':
    m.apply(init_normal)
  elif init_m == 'xaiver uniform':
    m.apply(init_xuniform)
  elif init_m == 'xavier normal':
    m.apply(init_xnormal)

# Best models from cross validation

In [15]:
best_models = [
               {'n_hidden': 400, 'n_layers': 8, 'act_fun': 'LeakyReLU', 'init_method': 'xavier uniform'},
               {'n_hidden': 400, 'n_layers': 4, 'act_fun': 'ReLU', 'init_method': 'xavier uniform'},
               {'n_hidden': 400, 'n_layers': 4, 'act_fun': 'LeakyReLU', 'init_method': 'xavier uniform'},
               {'n_hidden': 400, 'n_layers': 6, 'act_fun': 'LeakyReLU', 'init_method': 'xavier uniform'},
               {'n_hidden': 600, 'n_layers': 4, 'act_fun': 'LeakyReLU', 'init_method': 'xavier uniform'}
]

# Training

In [16]:
input_size = 7
output_size = 1
batch_size = 1208
epochs = 125
lr = 1e-4

loss_fn = nn.MSELoss()

In [17]:
class OptDataset(Dataset):

  def __init__(self, X, y):
    self.X = X
    self.y = y

  def __getitem__(self, idx):
    return self.X[idx], self.y[idx]

  def __len__(self):
    return len(self.X)

In [18]:
def evaluate(model, loss_fn, X, y):
  model.eval()
  with torch.no_grad():
    out = model(X.to(device))
    loss = loss_fn(out, y.to(device))
    return loss.item()

In [19]:
def mape_loss(y_pred, y):
  return sum([np.abs(y_i - hy_i) / y_i for y_i, hy_i in zip(y_pred, y)]) / len(y_pred)

In [20]:
def get_mape_loss(model, X, y):
  model.eval()
  with torch.no_grad():
    out = model(X.to(device))
    loss = mape_loss(
        out.squeeze().cpu().detach().numpy(), 
        y.squeeze().cpu().detach().numpy())
    return loss

In [21]:
def test_models(
  models_dict,
  epochs,
  batch_size,
  X_train,
  y_train,
  X_val,
  y_val,
  X_test,
  y_test,
  loss_fn
):
  testing_results = pd.DataFrame(columns=
                               ['hidden_size',
                                'n_layers',
                                'act_fun',
                                'init_methods',
                                'mean_val_result',
                                'std_val_result',
                                'test_mse',
                                'test_mae',
                                'test_rmse',
                                'test_mape'])

  for model_dict in models_dict:
    model = Net(input_size,
                output_size, 
                model_dict['n_hidden'], 
                model_dict['n_layers'], 
                model_dict['act_fun']).to(device)
    init_weights(model, model_dict['init_method'])
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)

    validation_losses = []
    test_res = {
        'hidden_size': model_dict['n_hidden'],
        'n_layers': model_dict['n_layers'],
        'act_fun': model_dict['act_fun'],
        'init_methods': model_dict['init_method']
    }

    print('Model: ', test_res)

    for epoch in range(epochs):
      model.train()
      total_loss = 0
      start_time = time.time()
      i = 0

      for batch, batch_labels in DataLoader(OptDataset(X_train, y_train), batch_size):
        out = model(batch.to(device))
        optimizer.zero_grad()

        loss = loss_fn(out, batch_labels.to(device))
        total_loss += loss.item()
        loss.backward()
        optimizer.step()

        if i > 0 and i % 25 == 0:
          avg_loss = total_loss / 50
          elapsed = time.time() - start_time
          print('| Epoch {:3d} | {:5d}/{:5d} batches | lr {:2.5f} | ms/batch {:5.2f} | '
                  'loss {:5.8f}'.format(
              epoch, i, len(X_train) // batch_size+1, lr, elapsed * 1000 / 50,
              avg_loss))
          start_time = time.time()
          total_loss = 0
        
        i += 1

      validation_losses.append(evaluate(model, loss_fn, X_val, y_val))
    
    mse_test = evaluate(model, loss_fn, X_test, y_test)
    mae_loss = nn.L1Loss()
    mae_test = evaluate(model, mae_loss, X_test, y_test)
    mape_test = get_mape_loss(model, X_test, y_test)
    validation_losses = np.array(validation_losses)
    test_res['mean_val_result'] = validation_losses.mean()
    test_res['std_val_result'] = validation_losses.std()
    test_res['test_mse'] = mse_test
    test_res['test_mae'] = mae_test
    test_res['test_rmse'] = np.sqrt(mse_test)
    test_res['test_mape'] = mape_test

    testing_results = testing_results.append(test_res, ignore_index=True)

  return testing_results

In [22]:
testing_results = test_models(best_models,
            epochs,
            batch_size,
             X_train,
            y_train,
            X_val,
            y_val,
            X_test,
            y_test,
            loss_fn)

Model:  {'hidden_size': 400, 'n_layers': 8, 'act_fun': 'LeakyReLU', 'init_methods': 'xavier uniform'}
| Epoch   0 |    25/  338 batches | lr 0.00010 | ms/batch 16.94 | loss 0.29636818
| Epoch   0 |    50/  338 batches | lr 0.00010 | ms/batch 11.21 | loss 0.02504778
| Epoch   0 |    75/  338 batches | lr 0.00010 | ms/batch  9.64 | loss 0.00809468
| Epoch   0 |   100/  338 batches | lr 0.00010 | ms/batch 11.30 | loss 0.00451989
| Epoch   0 |   125/  338 batches | lr 0.00010 | ms/batch 11.26 | loss 0.00279375
| Epoch   0 |   150/  338 batches | lr 0.00010 | ms/batch  9.66 | loss 0.00175735
| Epoch   0 |   175/  338 batches | lr 0.00010 | ms/batch 11.29 | loss 0.00124702
| Epoch   0 |   200/  338 batches | lr 0.00010 | ms/batch 11.27 | loss 0.00088120
| Epoch   0 |   225/  338 batches | lr 0.00010 | ms/batch 10.32 | loss 0.00067185
| Epoch   0 |   250/  338 batches | lr 0.00010 | ms/batch 11.35 | loss 0.00057322
| Epoch   0 |   275/  338 batches | lr 0.00010 | ms/batch 11.33 | loss 0.00046

| Epoch   7 |   225/  338 batches | lr 0.00010 | ms/batch 11.25 | loss 0.00003220
| Epoch   7 |   250/  338 batches | lr 0.00010 | ms/batch 11.31 | loss 0.00007083
| Epoch   7 |   275/  338 batches | lr 0.00010 | ms/batch  9.71 | loss 0.00002875
| Epoch   7 |   300/  338 batches | lr 0.00010 | ms/batch 11.29 | loss 0.00006430
| Epoch   7 |   325/  338 batches | lr 0.00010 | ms/batch 11.30 | loss 0.00006287
| Epoch   8 |    25/  338 batches | lr 0.00010 | ms/batch 11.66 | loss 0.00005953
| Epoch   8 |    50/  338 batches | lr 0.00010 | ms/batch 11.35 | loss 0.00006013
| Epoch   8 |    75/  338 batches | lr 0.00010 | ms/batch  9.68 | loss 0.00003066
| Epoch   8 |   100/  338 batches | lr 0.00010 | ms/batch 11.26 | loss 0.00004956
| Epoch   8 |   125/  338 batches | lr 0.00010 | ms/batch 11.26 | loss 0.00005167
| Epoch   8 |   150/  338 batches | lr 0.00010 | ms/batch  9.68 | loss 0.00003457
| Epoch   8 |   175/  338 batches | lr 0.00010 | ms/batch 11.25 | loss 0.00003281
| Epoch   8 |   

| Epoch  15 |   125/  338 batches | lr 0.00010 | ms/batch 11.28 | loss 0.00006222
| Epoch  15 |   150/  338 batches | lr 0.00010 | ms/batch 11.38 | loss 0.00001800
| Epoch  15 |   175/  338 batches | lr 0.00010 | ms/batch 11.57 | loss 0.00008013
| Epoch  15 |   200/  338 batches | lr 0.00010 | ms/batch 10.15 | loss 0.00007408
| Epoch  15 |   225/  338 batches | lr 0.00010 | ms/batch 11.55 | loss 0.00002585
| Epoch  15 |   250/  338 batches | lr 0.00010 | ms/batch 11.41 | loss 0.00002254
| Epoch  15 |   275/  338 batches | lr 0.00010 | ms/batch  9.74 | loss 0.00001355
| Epoch  15 |   300/  338 batches | lr 0.00010 | ms/batch 11.39 | loss 0.00004015
| Epoch  15 |   325/  338 batches | lr 0.00010 | ms/batch 11.36 | loss 0.00005059
| Epoch  16 |    25/  338 batches | lr 0.00010 | ms/batch 11.83 | loss 0.00001719
| Epoch  16 |    50/  338 batches | lr 0.00010 | ms/batch  9.87 | loss 0.00002927
| Epoch  16 |    75/  338 batches | lr 0.00010 | ms/batch 11.52 | loss 0.00003065
| Epoch  16 |   

| Epoch  23 |    25/  338 batches | lr 0.00010 | ms/batch 10.21 | loss 0.00001345
| Epoch  23 |    50/  338 batches | lr 0.00010 | ms/batch 11.33 | loss 0.00001787
| Epoch  23 |    75/  338 batches | lr 0.00010 | ms/batch 11.38 | loss 0.00005500
| Epoch  23 |   100/  338 batches | lr 0.00010 | ms/batch  9.70 | loss 0.00001812
| Epoch  23 |   125/  338 batches | lr 0.00010 | ms/batch 11.30 | loss 0.00003799
| Epoch  23 |   150/  338 batches | lr 0.00010 | ms/batch 11.26 | loss 0.00002118
| Epoch  23 |   175/  338 batches | lr 0.00010 | ms/batch  9.72 | loss 0.00001604
| Epoch  23 |   200/  338 batches | lr 0.00010 | ms/batch 11.38 | loss 0.00002881
| Epoch  23 |   225/  338 batches | lr 0.00010 | ms/batch 11.48 | loss 0.00002532
| Epoch  23 |   250/  338 batches | lr 0.00010 | ms/batch 10.04 | loss 0.00001854
| Epoch  23 |   275/  338 batches | lr 0.00010 | ms/batch 11.37 | loss 0.00003902
| Epoch  23 |   300/  338 batches | lr 0.00010 | ms/batch 11.40 | loss 0.00001194
| Epoch  23 |   

| Epoch  30 |   250/  338 batches | lr 0.00010 | ms/batch 11.57 | loss 0.00000878
| Epoch  30 |   275/  338 batches | lr 0.00010 | ms/batch 11.37 | loss 0.00000604
| Epoch  30 |   300/  338 batches | lr 0.00010 | ms/batch  9.79 | loss 0.00001130
| Epoch  30 |   325/  338 batches | lr 0.00010 | ms/batch 11.34 | loss 0.00000843
| Epoch  31 |    25/  338 batches | lr 0.00010 | ms/batch 11.81 | loss 0.00000656
| Epoch  31 |    50/  338 batches | lr 0.00010 | ms/batch 11.37 | loss 0.00001596
| Epoch  31 |    75/  338 batches | lr 0.00010 | ms/batch 11.38 | loss 0.00001071
| Epoch  31 |   100/  338 batches | lr 0.00010 | ms/batch  9.77 | loss 0.00004950
| Epoch  31 |   125/  338 batches | lr 0.00010 | ms/batch 11.37 | loss 0.00000657
| Epoch  31 |   150/  338 batches | lr 0.00010 | ms/batch 11.33 | loss 0.00000874
| Epoch  31 |   175/  338 batches | lr 0.00010 | ms/batch  9.75 | loss 0.00003536
| Epoch  31 |   200/  338 batches | lr 0.00010 | ms/batch 11.34 | loss 0.00002524
| Epoch  31 |   

...

| Epoch  91 |   325/  338 batches | lr 0.00010 | ms/batch 11.57 | loss 0.00001052
| Epoch  92 |    25/  338 batches | lr 0.00010 | ms/batch 12.02 | loss 0.00001414
| Epoch  92 |    50/  338 batches | lr 0.00010 | ms/batch  9.87 | loss 0.00001101
| Epoch  92 |    75/  338 batches | lr 0.00010 | ms/batch 11.59 | loss 0.00000862
| Epoch  92 |   100/  338 batches | lr 0.00010 | ms/batch 11.59 | loss 0.00000865
| Epoch  92 |   125/  338 batches | lr 0.00010 | ms/batch 10.19 | loss 0.00000407
| Epoch  92 |   150/  338 batches | lr 0.00010 | ms/batch 11.65 | loss 0.00000376
| Epoch  92 |   175/  338 batches | lr 0.00010 | ms/batch 11.56 | loss 0.00001549
| Epoch  92 |   200/  338 batches | lr 0.00010 | ms/batch 11.75 | loss 0.00001290
| Epoch  92 |   225/  338 batches | lr 0.00010 | ms/batch  9.98 | loss 0.00001330
| Epoch  92 |   250/  338 batches | lr 0.00010 | ms/batch 11.52 | loss 0.00000403
| Epoch  92 |   275/  338 batches | lr 0.00010 | ms/batch 11.51 | loss 0.00000836
| Epoch  92 |   

| Epoch  99 |   225/  338 batches | lr 0.00010 | ms/batch 11.55 | loss 0.00000389
| Epoch  99 |   250/  338 batches | lr 0.00010 | ms/batch 10.00 | loss 0.00000271
| Epoch  99 |   275/  338 batches | lr 0.00010 | ms/batch 11.55 | loss 0.00000909
| Epoch  99 |   300/  338 batches | lr 0.00010 | ms/batch 11.52 | loss 0.00001245
| Epoch  99 |   325/  338 batches | lr 0.00010 | ms/batch 11.64 | loss 0.00001091
| Epoch 100 |    25/  338 batches | lr 0.00010 | ms/batch 12.04 | loss 0.00001829
| Epoch 100 |    50/  338 batches | lr 0.00010 | ms/batch  9.92 | loss 0.00000893
| Epoch 100 |    75/  338 batches | lr 0.00010 | ms/batch 11.52 | loss 0.00000289
| Epoch 100 |   100/  338 batches | lr 0.00010 | ms/batch 11.61 | loss 0.00000341
| Epoch 100 |   125/  338 batches | lr 0.00010 | ms/batch  9.92 | loss 0.00000702
| Epoch 100 |   150/  338 batches | lr 0.00010 | ms/batch 11.49 | loss 0.00000289
| Epoch 100 |   175/  338 batches | lr 0.00010 | ms/batch 11.48 | loss 0.00000280
| Epoch 100 |   

| Epoch 107 |   125/  338 batches | lr 0.00010 | ms/batch 11.18 | loss 0.00000611
| Epoch 107 |   150/  338 batches | lr 0.00010 | ms/batch 11.19 | loss 0.00000597
| Epoch 107 |   175/  338 batches | lr 0.00010 | ms/batch  9.61 | loss 0.00000346
| Epoch 107 |   200/  338 batches | lr 0.00010 | ms/batch 11.12 | loss 0.00000731
| Epoch 107 |   225/  338 batches | lr 0.00010 | ms/batch 11.21 | loss 0.00000330
| Epoch 107 |   250/  338 batches | lr 0.00010 | ms/batch  9.60 | loss 0.00000278
| Epoch 107 |   275/  338 batches | lr 0.00010 | ms/batch 11.13 | loss 0.00000608
| Epoch 107 |   300/  338 batches | lr 0.00010 | ms/batch 11.20 | loss 0.00000378
| Epoch 107 |   325/  338 batches | lr 0.00010 | ms/batch  9.69 | loss 0.00000483
| Epoch 108 |    25/  338 batches | lr 0.00010 | ms/batch 10.04 | loss 0.00001731
| Epoch 108 |    50/  338 batches | lr 0.00010 | ms/batch 11.12 | loss 0.00000531
| Epoch 108 |    75/  338 batches | lr 0.00010 | ms/batch 11.14 | loss 0.00000483
| Epoch 108 |   

| Epoch 115 |    25/  338 batches | lr 0.00010 | ms/batch 11.49 | loss 0.00000554
| Epoch 115 |    50/  338 batches | lr 0.00010 | ms/batch 11.20 | loss 0.00003255
| Epoch 115 |    75/  338 batches | lr 0.00010 | ms/batch  9.60 | loss 0.00000704
| Epoch 115 |   100/  338 batches | lr 0.00010 | ms/batch 11.12 | loss 0.00000313
| Epoch 115 |   125/  338 batches | lr 0.00010 | ms/batch 11.13 | loss 0.00000261
| Epoch 115 |   150/  338 batches | lr 0.00010 | ms/batch  9.59 | loss 0.00000218
| Epoch 115 |   175/  338 batches | lr 0.00010 | ms/batch 11.10 | loss 0.00000218
| Epoch 115 |   200/  338 batches | lr 0.00010 | ms/batch 11.25 | loss 0.00000483
| Epoch 115 |   225/  338 batches | lr 0.00010 | ms/batch 11.20 | loss 0.00000847
| Epoch 115 |   250/  338 batches | lr 0.00010 | ms/batch  9.58 | loss 0.00000619
| Epoch 115 |   275/  338 batches | lr 0.00010 | ms/batch 11.14 | loss 0.00001278
| Epoch 115 |   300/  338 batches | lr 0.00010 | ms/batch 11.17 | loss 0.00002686
| Epoch 115 |   

| Epoch 122 |   250/  338 batches | lr 0.00010 | ms/batch 11.16 | loss 0.00000359
| Epoch 122 |   275/  338 batches | lr 0.00010 | ms/batch  9.60 | loss 0.00000545
| Epoch 122 |   300/  338 batches | lr 0.00010 | ms/batch 11.13 | loss 0.00000536
| Epoch 122 |   325/  338 batches | lr 0.00010 | ms/batch 11.15 | loss 0.00000341
| Epoch 123 |    25/  338 batches | lr 0.00010 | ms/batch 11.50 | loss 0.00002980
| Epoch 123 |    50/  338 batches | lr 0.00010 | ms/batch 11.21 | loss 0.00001503
| Epoch 123 |    75/  338 batches | lr 0.00010 | ms/batch  9.57 | loss 0.00000348
| Epoch 123 |   100/  338 batches | lr 0.00010 | ms/batch 11.13 | loss 0.00000343
| Epoch 123 |   125/  338 batches | lr 0.00010 | ms/batch 11.20 | loss 0.00000398
| Epoch 123 |   150/  338 batches | lr 0.00010 | ms/batch  9.61 | loss 0.00000428
| Epoch 123 |   175/  338 batches | lr 0.00010 | ms/batch 11.10 | loss 0.00000364
| Epoch 123 |   200/  338 batches | lr 0.00010 | ms/batch 11.15 | loss 0.00000272
| Epoch 123 |   

In [23]:
testing_results

,hidden_size,n_layers,act_fun,init_methods,mean_val_result,std_val_result,test_mse,test_mae,test_rmse,test_mape
0,400,8,LeakyReLU,xavier uniform,0.000054,0.000105,0.000009,0.002469,0.003063,-0.008389
1,400,4,ReLU,xavier uniform,0.000053,0.000089,0.000007,0.001971,0.002562,-0.004358
2,400,4,LeakyReLU,xavier uniform,0.000045,0.000091,0.000037,0.005260,0.006060,-0.000668
3,400,6,LeakyReLU,xavier uniform,0.000046,0.000084,0.000007,0.002200,0.002719,-0.000346
4,400,8,LeakyReLU,xavier uniform,0.000049,0.000078,0.000019,0.003483,0.004334,-0.036244


In [24]:
testing_results.to_csv('../results/testing_results_trinomial.csv')